In [7]:
from FundData import *
from helper import Helper
import pprint, json
import fitz

dir_path = r"C:\Users\Kaustubh.keny\OneDrive - Cogencis Information Services Ltd\Documents\mywork-repo"
#dir_path = r"C:\Users\rando\OneDrive\Documents\mywork-repo"
fund_path = r"C:\Users\Kaustubh.keny\OneDrive - Cogencis Information Services Ltd\Documents\Dec 24"
#fund_path = r'C:\Users\rando\OneDrive\Documents\Dec 24'

dry_path = r'\output\DryRun.pdf'
fin_path = r'\files\financial_indices.xlsx'


mutual_fund = Helper.get_fund_paths(fund_path)

In [251]:
def extract_data_relative_line(path: str, line_x: float, side: str):
    doc = fitz.open(path)
    pages = doc.page_count

    final_list = []

    for pgn in range(pages):
        page = doc[pgn]

        blocks = page.get_text("dict")["blocks"]
        sorted_blocks = sorted(blocks, key=lambda x: (x["bbox"][1], x["bbox"][0]))
        extracted_blocks = []

        # Keep track of blocks to avoid duplicates
        added_blocks = set()

        for block in sorted_blocks:
            block_id = id(block)  # Unique identifier for the block

            for line in block.get("lines", []):
                for span in line.get("spans", []):
                    origin = span["origin"]
                    x0, _ = origin

                    # Check the side condition
                    if side == "left" and x0 < line_x and block_id not in added_blocks:
                        extracted_blocks.append(block)
                        added_blocks.add(block_id)  # Mark block as added
                    elif side == "right" and x0 > line_x and block_id not in added_blocks:
                        extracted_blocks.append(block)
                        added_blocks.add(block_id)  # Mark block as added

      
        final_list.append({
            "pgn": pgn,
            "blocks": extracted_blocks
        })

    doc.close()

    return final_list

def extract_spans(data):
    final_span = []
    for pgn,blocks in enumerate(data):
        spans = []
        for block in blocks.get('blocks',[]):
            for line in block.get("lines", []):
                for span in line.get("spans", []):
                    if round(span['size']) in range(5,8) and span['color'] in [-15445130,-14590595] and span['font'] in ['Frutiger-Bold']:
                        print(span['text'], span['size'])
        final_span.append(spans)
    return final_span

def get_proper_fund_names(path:str,pages:list):
        
        doc = fitz.open(path)
        final_fund_names = dict()
        
        for pgn in range(doc.page_count):
            fund_names = ''
            if pgn in pages:
                page = doc[pgn]            
                blocks = page.get_text("dict")['blocks']
                
                sorted_blocks = sorted(blocks,key=lambda k:(k['bbox'][1],k['bbox'][0]))
                for count,block in enumerate(sorted_blocks):
                    for line in block.get("lines", []):
                        for span in line.get("spans", []):
                            text = span['text'].strip()
                            if count in range(0,2): #contains fund name        
                                fund_names += f'{text} '
            if matches:= re.search(r'\bMahindra.*?(Fund|ETF|EOF|FOF|FTF|Path)\b', fund_names, re.IGNORECASE):           
                final_fund_names[pgn] = matches.group()
            else:
                final_fund_names[pgn] = ''

        return final_fund_names

In [257]:
# Lines to be drawn (start and end points)
lines = [
    ((220, 0), (220, 812)),# Vertical line
    #((420, 0), (420, 1000))
]

rectangles = [(0,5,180,812)]

sample_path  = mutual_fund['Quant Mutual Fund']
pages = [i for i in range(1,180,1)]
Helper.draw_lines_on_pdf(sample_path, lines, rectangles, pages, dir_path +dry_path)
# data = Helper.get_all_pdf_data(sample_path)

Modified PDF saved to: C:\Users\Kaustubh.keny\OneDrive - Cogencis Information Services Ltd\Documents\mywork-repo\output\DryRun.pdf


In [258]:
relative_data = extract_data_relative_line(sample_path,180,"left")

In [259]:
relative_data

[{'pgn': 0,
  'blocks': [{'number': 5,
    'type': 0,
    'bbox': (155.39199829101562,
     707.5244750976562,
     453.685791015625,
     763.1624755859375),
    'lines': [{'spans': [{'size': 55.63800048828125,
        'flags': 20,
        'font': 'Roboto-Bold',
        'color': -65794,
        'ascender': 0.75,
        'descender': -0.25,
        'text': 'FACTSHEET',
        'origin': (155.39199829101562, 749.2529907226562),
        'bbox': (155.39199829101562,
         707.5244750976562,
         453.685791015625,
         763.1624755859375)}],
      'wmode': 0,
      'dir': (1.0, 0.0),
      'bbox': (155.39199829101562,
       707.5244750976562,
       453.685791015625,
       763.1624755859375)}]}]},
 {'pgn': 1, 'blocks': []},
 {'pgn': 2,
  'blocks': [{'number': 1,
    'type': 0,
    'bbox': (13.680000305175781,
     15.689972877502441,
     15.930960655212402,
     25.649972915649414),
    'lines': [{'spans': [{'size': 9.960000038146973,
        'flags': 16,
        'font': 'Cali